# Reconstructing morphological structure of layers from Allen data
See [this file](https://www.dropbox.com/sh/w5u31m3hq6u2x5m/AAD34a7F63f7nRYUq2vLR3sha/build_model/column_structure.xlsx?dl=0) or excel file in morphologies folder for model parameters

In [1]:
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm

import brian2 as b2

import copy

from allensdk.core.cell_types_cache import CellTypesCache
from allensdk.core.cell_types_cache import CellTypesCache
from allensdk.api.queries.cell_types_api import CellTypesApi
from allensdk.core.cell_types_cache import ReporterStatus as RS

import neurom as nm
from neurom import viewer

np.random.seed(42)

# Morphological parameters

In [2]:
# Relative percentage of E/I cells
frac_exc = .85
frac_inh = 1.-frac_exc

# Layer median thickness (um)
layer_thickness = {1 : 100,
                  2 : 210,
                  4 : 120,
                  5 : 220,
                  6 : 200}

# Cell counts per layer (scaled)
density_scale = .001
layer_density = {1 : 0 * density_scale,
                2 : 65949 * density_scale,
                4 : 53837 * density_scale,
                5 : 39480 * density_scale,
                6 : 67296 * density_scale}

# Layer morphotypes {layer : [(type, fraction)]}
layer_morphology = {1 : [('NO CELLS', .0)],
                    2 : [('Cux2', 1.)],
                    4 : [('Scnn1a', .3),
                         ('Rorb', .25),
                         ('Nr5a1', .12)],
                    5 : [('Rbp4', 1.)],
                    6 : [('Ntsr1', .8)]}

# Querying Allen DBs for morphological files

In [3]:
# Initiates the CellTypes constructor
morpho_path = '../morphologies/'
ctc = CellTypesCache(morpho_path) 

# Get all reconstructed mice neurons
cells = ctc.get_cells(require_reconstruction = True, 
                      species=[CellTypesApi.MOUSE])
print('Found %s cells in Allen DB' % len(cells))

Found 485 cells in Allen DB


In [12]:
# Filter cells to keep only V1 neurons with a specific names
layer_dict = {1 : [], 2 : [], 4 : [], 5 : [], 6 : []}
for cell in tqdm(cells, total = len(cells), desc = 'Fetching cells...') :
    if cell['structure_area_abbrev'] == 'VISp':
        layer = int(cell['structure_layer_name'][0]) #[0] eliminates the 2/3, 6a, 4b...
        morphotypes = layer_morphology[layer] # get the layer morphotypes names
        for types in morphotypes :
            if types[0] in cell['name'] :
                morpho = ctc.get_reconstruction(specimen_id = cell['id'],
                                            file_name = morpho_path + cell['name']  + '.swc')
                layer_dict[layer].append([cell['name'], cell['id'], morpho, morpho_path + cell['name']  + '.swc' ])
    else :
        pass

Fetching cells...: 100%|████████████████████████████████████████████████████████████| 485/485 [00:03<00:00, 143.84it/s]


# Creating an array of correct proportion of morphotypes for each layer

In [33]:
layer_array = {1 : [], 2 : [], 4 : [], 5 : [], 6 : []}
for layer, items in tqdm(layer_dict.items(), total = len(layer_dict), desc = 'Loading and shuffling morphologies') :
    _ = []
    for morph_tuples in layer_morphology[layer] :
        name_morpho = morph_tuples[0]
        fract_morpho = morph_tuples[1]
        
        density = layer_density[layer]
        n_cells = fract_morpho * density
        
        morpholist = [nm.load_neuron(x[-1]) for x in layer_dict[layer] if name_morpho in x[0]]
        morph_array = np.resize(morpholist, int(n_cells))
        np.random.shuffle(morph_array)
        
        _.append(morph_array)
    layer_array[layer].append(np.concatenate(_))

Loading and shuffling morphologies: 100%|████████████████████████████████████████████████| 5/5 [00:02<00:00,  2.11it/s]


# Distributing center of mass in a layer volume
### ATM with a noisy grid, but later with optimal packing or sphere packing
### Note : SVC files coordinates are in micrometers

In [ ]:
w,h = 1000, 1000 #um, size of the volume

In [56]:
neurontest = layer_array[2][0][0]

In [67]:
center_of_mass = [np.mean(neurontest.points[:,0]),
                 np.mean(neurontest.points[:,1]),
                 np.mean(neurontest.points[:,2]),]

In [68]:
center_of_mass

[366.76884697336567, 323.8611683884854, 36.08622744148507]